Last Update: Jan 30, 2024

In [118]:
import pandas as pd
from IPython.display import display

import urllib.parse as urlparse
from urllib.parse import parse_qs

import os
import sys

from datetime import datetime

In [119]:
# timestamp when running the code
date_code_excute = datetime.today().strftime("%Y%m%d")

In [120]:
# function for website & utm split
def website_utm_split (url, type):

    source_type = {"website":"", 
                   "utm_source": "", 
                   "utm_medium": "", 
                   "utm_campaign": "", 
                   "utm_content":"", 
                   "utm_term":"", 
                   "utm_id":""}

    # if url is empty
    if url == "":
        return source_type[type]
    
    # if url is not string
    if not isinstance(url, str):
        return source_type[type]

    # parse url
    url_split = urlparse.urlparse(url)

    # extract website
    website = url_split[0] + "://" + url_split[1] + url_split[2]
    source_type["website"] = website

    # extract utm source

    # if utm is empty
    if url_split.query == "":
        return source_type[type]

    # if utm is not empty
    utm_split = url_split.query.split("&")

    for i in utm_split:
        
        utm_type_para = i.split("=")

        # utm type
        utm_type = utm_type_para[0]

        # utm parameter
        utm_para = utm_type_para[1]

        source_type[utm_type] = utm_para
    
    return source_type[type]

In [121]:
# function to compile all csv in the file based on the first columns and to save the compiled output for the ease of checking
def compile_csv(folder_path, first_col, destination_folder, output_filename):

    """
    folder_path refers to the folder that contains all the csv that you wish to compile \n
    first_col refers to the name of the first column that you wish to detect \n
    destination_folder refers to the folder that you wish to save the compiled csv at (Suggest to save in the main working folder, not the csv compilation folder) \n
    output_filename refers to the filename for the compiled csv file (Remember to include the extension .csv)
    """

    # get all files in the folder
    all_files = os.listdir(folder_path)

    # filter out non-CSV files
    csv_files = [f for f in all_files if f.endswith(".csv")]

    # create a list to hold the dataframes
    df_list = []

    for csv in csv_files:
        file_path = os.path.join(folder_path, csv)
        print(f"{csv}")
    
        col_name = ""

        try:
            i = 0 
            # To identify which row to skip
            while col_name != first_col:            
                df = pd.read_csv(file_path, skiprows=i, nrows=1)
                col_name = df.columns[0]
                i += 1

            # Try reading the file using default UTF-8 encoding
            df = pd.read_csv(file_path, skiprows=i)

            print(f"Skipped {i} rows.")
            print(f"No of rows = {df.shape[0]}; No of columns = {df.shape[1]}")
            df_list.append(df)

        except UnicodeDecodeError:
            try:
                i = 0 
                # To identify which row to skip
                while col_name != first_col:
                    df = pd.read_csv(file_path, sep='\t', encoding='utf-16', skiprows=i, nrows=1)
                    col_name = df.columns[0]
                    i += 1

                # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
                df = pd.read_csv(file_path, sep='\t', encoding='utf-16', skiprows=i)
                
                print(f"Skipped {i} rows.")
                print(f"No of rows = {df.shape[0]}; No of columns = {df.shape[1]}")
                df_list.append(df)

            except Exception as e:
                print(f"Could not read file {csv} because of error: {e}")

        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")

    # Concatenate all data into one DataFrame
    df_compile = pd.concat(df_list, ignore_index=True)

    # Export as csv
    df_compile.to_csv(os.path.join(destination_folder, date_code_excute + "-" + output_filename), index=False)

    print(f"Total no of row = {df_compile.shape[0]}")

    return df_compile

In [122]:
# Objective keyword mapping

campaign_obj = {"Awareness":["Brand awareness"], 
                "Consideration":["Website visits", "Engagement", "Video views", "Messaging"],
                "Conversions":["Lead generation", "Talent leads", "Website conversions", "Job applicants"]}

In [123]:
# function for objective keyword mapping
def remap_campaign_obj(row):
    stage = row['Stage']

    for key, obj in campaign_obj.items():
        if stage in obj:
            return key

    raise ValueError(f"Invalid Objective Found {stage}. Please check the export CSV file.")

Update `main_folder_path` (Path to your main working folder)

In [124]:
main_folder_path = r"C:\Users\WeiZhenLim\OneDrive - 2X LLC\Work\013-016, 033 Televerde\Televerde Ad Optimization\02 Televerde LinkedIn Campaign Manager Export - Method 2"

#### Compile and Clean Up Ads Performance Report

Things to take note for the following two code blocks:

<u> First Code Block </u>
1. Replace the `folder_path` with the folder that contains all the **CSV** files that you wished to compile
2. Replace the `first_col` with the first column name of the data you wish to read 

<u> Output of Second Code Block </u> 
1. Check whether the number of row read matches with your files

In [125]:
# Replace with your folder's path
folder_path = r"C:\Users\WeiZhenLim\OneDrive - 2X LLC\Work\013-016, 033 Televerde\Televerde Ad Optimization\02 Televerde LinkedIn Campaign Manager Export - Method 2\LinkedIn Ads Report Compilation"

# Column name for the first column
first_col = "Start Date (in UTC)"

In [126]:
# Compile the CSV
df_report = compile_csv(folder_path, first_col, main_folder_path, "Ads Performance Report Compilation.csv")

01 Jan 2023 - May 2023.csv
Skipped 5 rows.
No of rows = 861; No of columns = 88
02 June 2023 - Nov 2023.csv
Skipped 5 rows.
No of rows = 2575; No of columns = 88
03 Dec 2023 - Jan 24 2024.csv
Skipped 5 rows.
No of rows = 39; No of columns = 88
Total no of row = 3475


In [127]:
# column name to subset and rename

report_ori_col = ["Campaign Group Name", "Campaign Group ID", "Campaign Name", "Campaign ID", 
                  "Creative Name", "Ad ID", "Campaign Objective", "Creative Status", 
                  "Start Date (in UTC)", "Click URL", "Ad Introduction Text", "Ad Headline"]

# Take note that Start Date (in UTC) and Click URL col is not rename as preprocessing is still require
report_rename_col = ["Campaign Group", "Campaign Group ID", "Campaign name", "Campaign ID", 
                     "Ad Name", "Ad ID", "Stage", "Ad status", 
                     "Start Date (in UTC)", "Click URL", "Introduction Text", "Headline Text"]

Check whether the status for the campaign group, campaign, and creative/ad is correct

In [128]:
# Check Status
df_report[['Campaign Group Status', 'Campaign Status', 'Creative Status']].apply(lambda col: col.unique())

Campaign Group Status               [Paused]
Campaign Status          [Completed, Paused]
Creative Status             [Active, Paused]
dtype: object

Check whether the number of campaign group, campaign ID, and Ad ID tally with your export

In [129]:
# Check the number of Campaign Groups, Campaign, and Ads
df_report[['Campaign Group ID', 'Campaign ID', 'Ad ID']].apply(lambda col: col.nunique())

Campaign Group ID     2
Campaign ID          21
Ad ID                81
dtype: int64

Only take the rows with Clicks > 0

In [130]:
print(f"No of rows before exclude Clicks = '0': {df_report.shape[0]}")

df_report = df_report[df_report['Clicks'] > 0]

print(f"No of rows after exclude Clicks = '0': {df_report.shape[0]}")

No of rows before exclude Clicks = '0': 3475
No of rows after exclude Clicks = '0': 3333


In [131]:
# Subset the required field and rename the column
df_report = df_report[report_ori_col]

df_report.columns = report_rename_col

In [132]:
df_report.head(1)

,Campaign Group,Campaign Group ID,Campaign name,Campaign ID,Ad Name,Ad ID,Stage,Ad status,Start Date (in UTC),Click URL,Introduction Text,Headline Text
0,2023 Demand Gen Campaign Group,632500803,Net New General - February 2023,220750463,ICP-February.V1,231947133,Website visits,Active,2/18/2023,https://televerde.com/7-defining-characteristi...,Customers want to see socially responsible com...,Don't just talk the talk. Walk the talk.


#### Get the Launch date and Completed date

In [133]:
ads_date = df_report[['Ad ID', 'Start Date (in UTC)']].copy()
ads_date['Date'] = pd.to_datetime(ads_date['Start Date (in UTC)'])
ads_date.head()

,Ad ID,Start Date (in UTC),Date
0,231947133,2/18/2023,2023-02-18
1,231947133,2/19/2023,2023-02-19
2,231947133,2/20/2023,2023-02-20
3,231947133,2/21/2023,2023-02-21
4,231947133,2/22/2023,2023-02-22


In [134]:
# Create aggregation to get the max and min
launch_comp_date = pd.pivot_table(ads_date, values='Date', index=['Ad ID'], aggfunc=['min', 'max']).reset_index()\

# Rename to the correct name
launch_comp_date.rename(columns={'min':'Launch date', 'max':'Completed date'}, inplace=True)

# Drop MultiIndex
launch_comp_date = launch_comp_date.droplevel(1, axis=1)

launch_comp_date

,Ad ID,Launch date,Completed date
0,231947133,2023-02-18,2023-02-28
1,231947273,2023-02-18,2023-02-28
2,231949233,2023-02-18,2023-02-28
3,231949273,2023-02-18,2023-02-28
4,231949853,2023-02-18,2023-02-28
...,...,...,...
76,310860716,2023-09-22,2023-11-15
77,310860736,2023-09-22,2023-11-15
78,324756683,2023-11-16,2023-12-12
79,324783513,2023-11-16,2023-12-12


Update `ads_date_file` (Rmb to include .csv)

In [135]:
ads_date_file = "Launch date and Completed date check.csv"

In [136]:
ads_date.to_csv(os.path.join(main_folder_path, date_code_excute + "-" + ads_date_file))

#### Process Ads Performance Report for utm source & remap stage

In [137]:
df_report.drop('Start Date (in UTC)', axis=1, inplace=True)
df_report.drop_duplicates(inplace=True)

In [138]:
# Extract for website and utm source
df_report['Website URL'] = df_report['Click URL'].apply(lambda x: website_utm_split(x, "website"))
df_report['utm_source'] = df_report['Click URL'].apply(lambda x: website_utm_split(x, "utm_source"))
df_report['utm_medium'] = df_report['Click URL'].apply(lambda x: website_utm_split(x, "utm_medium"))
df_report['utm_campaign'] = df_report['Click URL'].apply(lambda x: website_utm_split(x, "utm_campaign"))
df_report['utm_content'] = df_report['Click URL'].apply(lambda x: website_utm_split(x, "utm_content"))
df_report['utm_term'] = df_report['Click URL'].apply(lambda x: website_utm_split(x, "utm_term"))
df_report['utm_id'] = df_report['Click URL'].apply(lambda x: website_utm_split(x, "utm_id"))

In [139]:
# Remap for stage
df_report['Stage'] = df_report.apply(remap_campaign_obj, axis=1)

In [140]:
# Drop the Click URL columns
df_report.drop('Click URL', axis=1, inplace=True)
df_report.columns

Index(['Campaign Group', 'Campaign Group ID', 'Campaign name', 'Campaign ID',
       'Ad Name', 'Ad ID', 'Stage', 'Ad status', 'Introduction Text',
       'Headline Text', 'Website URL', 'utm_source', 'utm_medium',
       'utm_campaign', 'utm_content', 'utm_term', 'utm_id'],
      dtype='object')

#### Compile Ads LinkedIn Bulk Export (Sometimes the Ads Bulk Export does not work, try with Campaign instead)

Use the `compile_csv` function above to compile the bulk export (For cases where your client has multiple ad accounts)

In [ ]:
"""
campaign_bulk_folder = r"C:\Users\WeiZhenLim\OneDrive - 2X LLC\Work\013-016, 033 Televerde\Televerde Ad Optimization\03 Broadcom LinkedIn Campaign Manager Export\LinkedIn Campaign Bulk Report Compilation"

compile_csv(campaign_bulk_folder, "*Account ID", main_folder_path, "LinkedIn Bulk Export.csv")

"""

Update `linkedin_bulk_export_file` (Rmb to include .csv)

In [141]:
linkedin_bulk_export_file = "20240124-LinkedIn Bulk Export.csv"

In [142]:
# function to return the correct list of columns
def ad_or_campaign(group_type):

    if group_type not in ["Ad", "Campaign"]:
        raise ValueError("The group_type must be either Ad or Campaign")

    if group_type == "Ad":
        return [['Creative ID', 'Ad format'], ['Ad ID', 'Ad type']]
    else:
        return [['Campaign ID', '*Ad Format'], ['Campaign ID', 'Ad type']]

In [143]:
# function for Campaign or Ads Bulk Export
def ads_or_campaign_bulk(folder_path, filename, group_type):

    """
    folder_path refers to the folder that contains the bulk export \n
    filename refers to the filename of the bulk export csv (Remember to include the .csv extension) \n
    group_type refers to the type of CSV bulk export from LinkedIn. It can only be either "Ad" or "Campaign"
    """

    if group_type not in ["Ad", "Campaign"]:
        raise ValueError("The group_type must be either Ad or Campaign")

    try:

        df_bulk = pd.read_csv(os.path.join(folder_path, filename))

    except UnicodeDecodeError:
        try:

            df_bulk = pd.read_csv(os.path.join(folder_path, filename), sep='\t', encoding='utf-16')

        except Exception as e:
            print(f"Could not read file {filename} because of error: {e}")

    except Exception as e:
        print(f"Could not read file {filename} because of error: {e}")
    
    ori_col_name = ad_or_campaign(group_type)[0]

    new_col_name = ad_or_campaign(group_type)[1]

    df_bulk = df_bulk[ori_col_name]

    df_bulk.columns = new_col_name
        
    return df_bulk

In [144]:
df_bulk = ads_or_campaign_bulk(main_folder_path, linkedin_bulk_export_file, "Ad") # Comment this block if Campaign Bulk Export is used

# df_bulk = ads_or_campaign_bulk(main_folder_path, linkedin_bulk_export_file, "Campaign") # Uncomment this block if Campaign Bulk Export is used

In [145]:
df_bulk.head()

,Ad ID,Ad type
0,310837536,Single image ad
1,310827936,Single image ad
2,310860736,Single image ad
3,235609313,Single image ad
4,277644993,Single image ad


To check the number of ads you had on LinkedIn

In [146]:
# Check Stats
df_bulk['Ad ID'].nunique() # Comment this block if Campaign Bulk Export is used
# df_bulk['Campaign ID'].nunique() # Uncomment this block if Campaign Bulk Export is used

60

To check whether the ad types matches with the ads types that you had on LinkedIn

In [147]:
# Check Stats
df_bulk['Ad type'].unique()

array(['Single image ad'], dtype=object)

#### Join all dataframe together

In [148]:
display(df_report.head(1))
display(launch_comp_date.head(1))
display(df_bulk.head(1))

,Campaign Group,Campaign Group ID,Campaign name,Campaign ID,Ad Name,Ad ID,Stage,Ad status,Introduction Text,Headline Text,Website URL,utm_source,utm_medium,utm_campaign,utm_content,utm_term,utm_id
0,2023 Demand Gen Campaign Group,632500803,Net New General - February 2023,220750463,ICP-February.V1,231947133,Consideration,Active,Customers want to see socially responsible com...,Don't just talk the talk. Walk the talk.,https://televerde.com/7-defining-characteristi...,,,,,,


,Ad ID,Launch date,Completed date
0,231947133,2023-02-18,2023-02-28


,Ad ID,Ad type
0,310837536,Single image ad


Remember to update the key for the second merge as Campaign ID (If you are using Campaign Bulk Export)

In [149]:
df_ads_final = pd.merge(left=df_report, right=launch_comp_date, how='left', on='Ad ID')
df_ads_final = pd.merge(left=df_ads_final, right=df_bulk, how='left', on='Ad ID') # Comment this if Campaign Bulk Export is used
# df_ads_final = pd.merge(left=df_ads_final, right=df_bulk, how='left', on='Campaign ID') # Uncomment this if Campaign Bulk Export is used
df_ads_final.head()

,Campaign Group,Campaign Group ID,Campaign name,Campaign ID,Ad Name,Ad ID,Stage,Ad status,Introduction Text,Headline Text,Website URL,utm_source,utm_medium,utm_campaign,utm_content,utm_term,utm_id,Launch date,Completed date,Ad type
0,2023 Demand Gen Campaign Group,632500803,Net New General - February 2023,220750463,ICP-February.V1,231947133,Consideration,Active,Customers want to see socially responsible com...,Don't just talk the talk. Walk the talk.,https://televerde.com/7-defining-characteristi...,,,,,,,2023-02-18,2023-02-28,Single image ad
1,2023 Demand Gen Campaign Group,632500803,Net New General - February 2023,220750463,ICP-February.V2,231947273,Consideration,Active,Keeping pace with digital demands is what will...,Technology is rapidly evolving. Are you evolvi...,https://televerde.com/6-ways-you-can-make-a-gr...,,,,,,,2023-02-18,2023-02-28,Single image ad
2,2023 Demand Gen Campaign Group,632500803,Net New Manufacturing - February 2023,220751003,ICP-February.V3,231949233,Consideration,Active,Modernization is the secret to getting ahead o...,Technology is rapidly evolving. Are you evolvi...,https://televerde.com/6-ways-you-can-make-a-gr...,,,,,,,2023-02-18,2023-02-28,Single image ad
3,2023 Demand Gen Campaign Group,632500803,Net New Manufacturing - February 2023,220751003,ICP-February.V4,231949273,Consideration,Active,"You're dealing with labor shortage, skills gap...",A positive customer experience is integral for...,https://televerde.com/5-smart-strategies-for-h...,,,,,,,2023-02-18,2023-02-28,Single image ad
4,2023 Demand Gen Campaign Group,632500803,Net New Software - February 2023,220751633,ICP-February.V5,231949853,Consideration,Active,Providing a superior customer experience could...,A positive customer experience is integral for...,https://televerde.com/5-smart-strategies-for-h...,,,,,,,2023-02-18,2023-02-28,Single image ad


In [150]:
# Exit if the number of columns is not exactly 20. Check the column naming
if df_ads_final.shape[1] != 20:
    sys.exit("Number of columns does not match with Airtable Column Naming. Please Check")

#### Convert into Airtable Format

Update `airtable_col_filename` (Rmb to include .csv)

In [151]:
airtable_col_filename = "Airtable Column.csv"

In [152]:
df_airtable = pd.read_csv(os.path.join(main_folder_path, airtable_col_filename))

In [153]:
df_final = pd.concat([df_airtable, df_ads_final])

C:\Users\WeiZhenLim\AppData\Local\Temp\ipykernel_12720\2786435345.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_airtable, df_ads_final])


In [154]:
# Exit if the number of columns does not match with Airtable. Check the column naming
if df_final.shape[1] != df_airtable.shape[1]:
    sys.exit("Number of columns does not match with Airtable. Please Check")

The following line of code is subjected to change as the following is inserted because LinkedIn could not export Conversation ad.

In [155]:
# Replace missing with "Conversation ad"
df_final['Ad type'] = df_final['Ad type'].apply(lambda x: x if x == "Single image ad" else "Conversation ad")
df_final['Ad type'].value_counts()

Ad type
Single image ad    60
Conversation ad    21
Name: count, dtype: int64

The following line of code is subjected to the size used for the campaign. Hence the logic will be different.

In [156]:
# Update size
df_final['Size'] = df_final['Ad type'].apply(lambda x: "1200x628" if x == "Single image ad" else "")
df_final['Size'].value_counts()

Size
1200x628    60
            21
Name: count, dtype: int64

The following line of code could be different depending on the LinkedIn Ads setting.

In [157]:
# Update Label and Platform
df_final['Label'] = "Show"
df_final['Platform'] = "LinkedIn"

#### Subset the compiled dataframe based on your requirement (If necessary)

#### Export to csv

Update the `output_filename` (Rmb to include .csv)

In [158]:
output_filename = "Televerde LinkedIn to Airtable.csv"

In [159]:
df_final.to_csv(os.path.join(main_folder_path, date_code_excute + "-" + output_filename), index=False)